# Урок 8

## Дисперсионный анализ. Логистическая регрессия.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### Задача 1

#### Объясняемая переменная y зависит от двух категориальных факторов `A` и `B`, причём фактор `A` имеет 3 уровня, а фактор `B` - 4 уровня. Результаты наблюдений приведены в таблице:

```
y = [[2.68, 3.29, 2.88, 4.45],
     [4.12, 4.96, 5.09, 5.22],
     [5.52, 4.50, 5.42, 5.29]
```
     
__С помощью двухфакторного дисперсионного анализа проверьте влияние каждого из факторов на переменную `y`.__

__Решение:__

Нулевые гипотезы:

$H_{0A}: \: \overline{y_{1 \ast}} = \dots = \overline{y_{m \ast}}\;$ (фактор _не значим_)

$H_{0B}: \: \overline{y_{\ast 1}} = \dots = \overline{y_{\ast k}}\;$ (фактор _не значим_)

Ниже следует блок вычислений оценки дисперсий через суммы квадратов отклонений и поиск Т-статискик для факторов `A` и `B`, чтобы оценить их значимость действительность гипотез.

In [13]:
y_a1 = np.array([2.68, 3.29, 2.88, 4.45])
y_a2 = np.array([4.12, 4.96, 5.09, 5.22])
y_a3 = np.array([5.52, 4.50, 5.42, 5.29])

y_b1 = np.array([2.68, 4.12, 5.52])
y_b2 = np.array([3.29, 4.96, 4.50])
y_b3 = np.array([2.88, 5.09, 5.42])
y_b4 = np.array([4.45, 5.22, 5.29])

m, k = 3, 4

y = np.concatenate([y_a1, y_a2, y_a3])
y

array([2.68, 3.29, 2.88, 4.45, 4.12, 4.96, 5.09, 5.22, 5.52, 4.5 , 5.42,
       5.29])

In [15]:
y_a1_mean = np.mean(y_a1)
y_a2_mean = np.mean(y_a2)
y_a3_mean = np.mean(y_a3)

y_a1_mean, y_a2_mean, y_a3_mean

(3.325, 4.8475, 5.1825)

In [16]:
y_b1_mean = np.mean(y_b1)
y_b2_mean = np.mean(y_b2)
y_b3_mean = np.mean(y_b3)
y_b4_mean = np.mean(y_b4)

y_b1_mean, y_b2_mean, y_b3_mean, y_b4_mean

(4.1066666666666665, 4.25, 4.463333333333334, 4.986666666666667)

In [17]:
y_mean = np.mean(y)
y_mean

4.451666666666666

In [20]:
s2_a = k * (((y_a1_mean - y_mean) ** 2) + ((y_a2_mean - y_mean) ** 2) + ((y_a3_mean - y_mean) ** 2))
s2_b = m * (((y_b1_mean - y_mean) ** 2) + ((y_b2_mean - y_mean) ** 2) + ((y_b3_mean - y_mean) ** 2) 
            + ((y_b4_mean - y_mean) ** 2))
s2_res = np.sum((y_a1 - y_a1_mean - y_b1_mean + y_mean) ** 2) + np.sum((y_a2 - y_a2_mean - y_b2_mean + y_mean) ** 2) 
+ np.sum((y_a3 - y_a3_mean - y_b3_mean + y_mean) ** 2)
s2_b, s2_a, s2_res

(1.338166666666669, 7.8407166666666654, 3.2591527777777762)

In [24]:
sigma2_a = s2_a / (m - 1)
sigma2_b = s2_b / (k - 1)
sigma2_res = s2_res / ((k - 1) * (m - 1))

sigma2_a, sigma2_b, sigma2_res

(3.9203583333333327, 0.4460555555555563, 0.5431921296296294)

__Считаем Т-статистики:__

In [35]:
F_h_a = sigma2_a / sigma2_res
F_h_b = sigma2_b / sigma2_res

n = k * m

f1_a = m - 1
f2_a = n - m

f1_b = k - 1
f2_b = n - k

alpha = 0.05

F_crit_a = stats.f.ppf(1 - alpha, f1_a, f2_a)

F_crit_b = stats.f.ppf(1 - alpha, f1_b, f2_b)

print('Проверка значимости фактора А')

if F_h_a > F_crit_a:
    print(f'Фактор А значим, т.е. гипотеза отвергается (T_A > T_crit, или {F_h_a} > {F_crit_a})')
else:
    print('Фактор А не значим, т.е. гипотеза принимается')
    
print('Проверка значимости фактора B')

if F_h_b < F_crit_b:
    print(f'Фактор B не значим, т.е. гипотеза принимается (T_B < T_crit, или {F_h_b} < {F_crit_b})')
else:
    print('Фактор B значим, т.е. гипотеза отвергается')

Проверка значимости фактора А
Фактор А значим, т.е. гипотеза отвергается (T_A > T_crit, или 7.217259086589479 > 4.25649472909375)
Проверка значимости фактора B
Фактор B не значим, т.е. гипотеза принимается (T_B < T_crit, или 0.8211745554187158 < 4.06618055135116)


### Задача 2

__С помощью критерия Стьюдента для двух независимых выборок проверьте гипотезу о равенстве среднего роста футболистов и хоккеистов, основываясь на результатах измерений:__
```
football_players = [173, 175, 180, 178, 177, 185, 183, 182]
hockey_players = [177, 179, 180, 188, 177, 172, 171, 184, 180]
```

__Решение:__

Нулевая гипотеза:

$H_{0A}: {\overline{X_1} =\overline{X_2}}$

Ниже следует блок вычислений для проверки гипотезы.

In [37]:
football_players = np.array([173, 175, 180, 178, 177, 185, 183, 182])
hockey_players = np.array([177, 179, 180, 188, 177, 172, 171, 184, 180])

n1 = football_players.shape[0]
n2 = hockey_players.shape[0]

mean1 = football_players.mean()
mean2 = hockey_players.mean()

sigma1 = football_players.std(ddof=1)
sigma2 = hockey_players.std(ddof=1)

T = (mean1 - mean2) / np.sqrt((sigma1 ** 2) / n1 + (sigma2 ** 2) / n2)
print('T_stat = ',T)

T_stat =  0.19928601397363732


In [38]:
t = stats.t.ppf(1 - alpha / 2, df = n1 + n2 - 2)
print('t_stat = ',t)

t_stat =  2.131449545559323


In [39]:
if -t < T < t:
    print('Гипотеза принимается')

Гипотеза принимается


### Задача 3

__Прикрепите ссылку на курсовой проект на `github` (см. выбор темы в ДЗ к Уроку 6).__

Будет к 8-му декабря.